In [36]:
% pylab notebook
import datacube
from datacube.storage import masking
from datacube import Datacube
from datetime import datetime
from skimage import exposure

from datacube_stats.statistics import GeoMedian
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

dc = datacube.Datacube()
# Replace '156' and 'cek156' with the path to your own home directory on the VDI
s2aws = Datacube(config='/home/547/ck9738/datacube-s2.conf')

Populating the interactive namespace from numpy and matplotlib


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Sentinal 2 A&B Geomedian notebook

what does this notebook do? 

In [37]:
def three_band_image_flat(ds, bands, time = 0, figsize = [10,10], projection = 'projected'):
    '''
    threeBandImage takes three spectral bands from a dataframe with no time dimention, and plots them on the RGB 
    bands of an image. 
    
    Last modified: March 2018
    Author: Mike Barnes
    Modified by: Claire Krause
    Inputs: 
    ds -   Dataset containing the bands to be plotted
    bands - list of three bands to be plotted
    
    Optional:
    time - Index value of the time dimension of ds to be plotted
    figsize - dimensions for the output figure
    projection - options are 'projected' or 'geographic'. To determine if the image is 
    in degrees or northings
    '''
    y, x = ds[bands[0]].shape
    rawimg = np.zeros((y,x,3), dtype = np.float32)
    for i, colour in enumerate(bands):
        rawimg[:,:,i] = ds[colour].values
    rawimg[rawimg == -999] = np.nan
    img_toshow = exposure.equalize_hist(rawimg, mask = np.isfinite(rawimg))
    fig = plt.figure(figsize = figsize)
    plt.imshow(img_toshow)
    ax = plt.gca()
    ax.set_title('plot', fontweight = 'bold', fontsize = 16)
    ax.set_xticklabels(ds.x.values)
    ax.set_yticklabels(ds.y.values)
    if projection == 'geographic':
        ax.set_xlabel('Longitude', fontweight = 'bold')
        ax.set_ylabel('Latitude', fontweight = 'bold')
    else:
        ax.set_xlabel('Eastings', fontweight = 'bold')
        ax.set_ylabel('Northings', fontweight = 'bold')


#set up peramiters for sentinal data loading

Ssensors= ['s2a_ard_granule', 's2b_ard_granule'] #pick the sentinal satelites you want

Sbands_of_int =['red','blue','green'] #pick the sentinal bands that you want, here i am just using visible light  

Squery = {
        'lat': (-30.755, -30.795),
        'lon': (144.880, 144.940),
        'output_crs': 'EPSG:3577',
        'resolution': (-10, 10),
        'time':('2017-06-01', '2018-01-01')
        }
# use s2b_ard_granule 	 for S2B
#data = s2aws.load(product='s2a_ard_granule', group_by='solar_day', **query)

In [38]:
#Temporal range, wavelengths/band and sensors of interest are defined for LANDSAT

Lsensors= ['ls5_nbart_albers','ls7_nbart_albers'] #pick the landsat satelites you want

Lbands_of_int =['red','blue','green', 'nir'] #pick the sentinal bands that you want, here i am just using visible light  

Lquery = {
        'lat': (-19.28, -19.32),
        'lon': (146.74, 146.78),
        'output_crs': 'EPSG:3577',
        'resolution': (-25, 25),
        'time':('2008-06-01', '2010-05-30')
        }

# this quiery retreives our data request entered in above cell

data_land = dc.load(product=Lsensors, measurements= Lbands_of_int, group_by='solar_day',  **Lquery)

data_land

TypeError: unhashable type: 'list'

data_sent = {}

for i in Ssensors:
    this_sensor = s2aws.load(product= i, group_by='solar_day', **Squery)
    
    #retreive projection infromation
    crs = this_sensor.crs
    crswkt = this_sensor.crs.wkt
    affine = this_sensor.affine
    
    #make a mask for full timeseries// if you aren't using a geomedian you may want this, but we don't need it
    #clear_pixels = this_sensor.pixel_quality == 1
    #clear_data = this_sensor.where(clear_pixels)
    
    data_sent[i] = this_sensor

#concatinate all of the sentinal data together into one xarray, ordered by time 

data_sent = xr.concat(data_sent.values(),'time')
data_sent = data_sent.sortby('time')
data_sent.attrs['crs'] = crs
data_sent.attrs['affin|e'] = affine

data_sent

In [ ]:
#try to load 10m resampled landsat data

#data_land = dc.load(product='ls8_nbart_albers', like=data_sent)

In [ ]:
data_land

In [ ]:
#cast the data to float numbers so geomedian calculation will work
#data_sent_flt = data_sent.astype(float,casting='safe')
data_land_flt = data_land.astype(float,casting='safe')

In [ ]:
#create geomedian for data
#Sentinal_gm=GeoMedian().compute(data_sent_flt)

In [ ]:
#create geomedian for data
lnds_8_gm=GeoMedian().compute(data_land_flt)
    

In [ ]:
#sent_NDVI =  (Sentinal_gm['nir1']-Sentinal_gm['red'])/(Sentinal_gm['nir1']+Sentinal_gm['red'])
land_NDVI =  (lnds_8_gm['nir']-lnds_8_gm['red'])/(lnds_8_gm['nir']+lnds_8_gm['red'])

In [ ]:
#plt.scatter(sent_NDVI.data.ravel(),land_NDVI.data.ravel(), alpha=0.1)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plt.title('Time Two NDVI')
ax.imshow(land_NDVI)
#plt.savefig('plots/land/ndvi_after_diffcolo.png')
plt.show()

In [ ]:
three_band_image_flat(lnds_8_gm, bands=['red','green','blue'])

In [ ]:
#three_band_image_flat(lnds_8_gm, bands=['red','green','blue'],)